In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from ast import literal_eval

In [2]:
# human_df = pd.read_csv("../data_pixtral_llava/val_set_full.csv", index_col=[0])
# human_df.shape

In [3]:
anno_df = pd.read_json("../data_pixtral_llava/output/multipleframes_val_set_full_pixtral_anno.jsonl", lines=True, orient="records")
anno_df.sample(2)

,mini_multiple_frames_frames-list,mini_multiple_frames_reason,prompt_entity_entity-name,prompt_entity_entity-gender,prompt_entity_sentiment,prompt_entity_sentiment-reason,image_url,uuid
207,"[Quality of life, Cultural identity]",The image depicts a cozy café with people enjo...,Lost Boy,None,neutral,The image depicts a casual and relaxed atmosph...,data_pixtral_llava/images/d11987cf-ac42-4d72-b...,d11987cf-ac42-4d72-b805-b9f6ea39e413
271,"[Capacity and resources, Quality of life]",The image shows a geographical area with a riv...,None,None,neutral,The image depicts a natural landscape with a r...,data_pixtral_llava/images/cbd6b761-a3c6-48a7-8...,cbd6b761-a3c6-48a7-836e-e248db08fa1d


In [4]:
def plot_image(img_urls, labels, preds):
    assert len(img_urls) == 4
    fig, ax = plt.subplots(1, 4, figsize=(20,5))
    for i, (url, label, pred) in enumerate(zip(img_urls, labels, preds)):
        image = plt.imread("../"+url)
        ax[i].imshow(image)
        ax[i].set_title(f"Label: {label}\nPred: {pred}")
        ax[i].axis("off")

def plot_image_title(img_url, explanation, title):
    image = plt.imread("../"+img_url)
    plt.imshow(image)
    plt.figtext(0.5, 0.01, explanation, wrap=True, horizontalalignment='center', fontsize=7)
    plt.title(f"Article Title: {title}")
    plt.axis("off")

### Frame improvements

In [5]:
human_df = pd.read_csv("../data_pixtral_llava/val_set_full.csv", index_col=[0])
anno_df = pd.read_json("../data_pixtral_llava/output/multipleframes_val_set_full_pixtral_anno.jsonl", lines=True, orient="records")
anno_df.set_index("uuid", inplace=True)

In [6]:
len(anno_df), len(human_df)

(300, 300)

In [7]:
anno_df.sample(2)

,mini_multiple_frames_frames-list,mini_multiple_frames_reason,prompt_entity_entity-name,prompt_entity_entity-gender,prompt_entity_sentiment,prompt_entity_sentiment-reason,image_url
uuid,,,,,,,
89f04435-4c85-4059-be08-5024cdee34ca,[Crime and punishment],"The image shows a police officer in uniform, w...",Broward Sheriff's Office,None,positive,The image shows a person in a uniform associat...,data_pixtral_llava/images/89f04435-4c85-4059-b...
69249b2f-dd5e-4590-b92d-3778c18bc888,"[Quality of life, None]",The image shows people resting in what appears...,Homeless individuals,Non-binary,Neutral,The image depicts homeless individuals resting...,data_pixtral_llava/images/69249b2f-dd5e-4590-b...


In [8]:
label_cols = ['mini_multiple_frames_frames-list']#, 'normal_multiple_frames_frame-list'] #normal_multiple_frames_frames-list
for col in label_cols:
    for i, row in anno_df[col].items():
        if isinstance(row, list):
            anno_df.at[i, col] = set([i.lower() for i in row])

In [9]:
human_df['merged_labels'] = human_df['merged_labels'].apply(lambda x: literal_eval(x))
human_df.shape

(300, 1)

In [10]:
merged_df = human_df.join(anno_df[label_cols+['image_url']], how="inner")
merged_df.shape

(300, 3)

In [11]:
for col in label_cols:
    merged_df[col] = merged_df[col].str.lower()

In [12]:
# results = {}
# for col in label_cols:
#         correct = 0
#         for i, row in merged_df.iterrows():
#                 if row[col] in row['merged_labels']:
#                         correct += 1
#         accuracy = correct / len(merged_df)
#         results[col] = accuracy
#         print(col, accuracy)

In [13]:
# pd.DataFrame(results, index=["accuracy"]).T.plot(kind="barh", figsize=(10, 5))

In [14]:
pred_df = anno_df[["mini_multiple_frames_frames-list","mini_multiple_frames_reason", 'image_url']]
pred_df.columns = ['pred_labels', 'pred_reason', 'image_url']
pred_df.shape

(300, 3)

In [15]:
human_df.columns = ['human_annotations']

In [16]:
dropped_df = pred_df['pred_labels'].dropna()
merged_df = pd.merge(dropped_df, human_df, left_index=True, right_index=True)
merged_df.head()


,pred_labels,human_annotations
uuid,,
e78308e9-8c49-4217-9bb2-ef93a4b82b90,"{security and defense, crime and punishment}","{security and defense, crime and punishment, f..."
6261799f-3a92-4805-bc56-3252dd645d89,"{security and defense, crime and punishment}",{security and defense}
80164efd-4678-4b00-92c6-eec6162e0cf1,{none},"{none, quality of life}"
e336c7fb-895c-4d87-a384-2b1e479cdb9c,{none},"{none, quality of life}"
0054ad1b-896e-42cf-a3fb-c9eedea17ec4,"{legality, constitutionality and jurisprudence...",{political}


In [17]:
merged_df['intersection'] = merged_df.apply(lambda x: x['pred_labels'].intersection(x['human_annotations']), axis=1)
merged_df['intersection_len'] = merged_df['intersection'].apply(len)
merged_df['correct'] = merged_df['intersection_len'] > 0
print(merged_df['correct'].sum()/len(dropped_df))
print("------------------")

0.78
------------------


### Removing model None values

In [18]:
# Filter out rows where the model prediction is 'None'
filtered_df = pred_df[pred_df['pred_labels'] != {'none'}]

# Perform the same calculation on the filtered DataFrame
dropped_df_filtered = filtered_df['pred_labels'].dropna()
merged_df_filtered = pd.merge(dropped_df_filtered, human_df, left_index=True, right_index=True)
merged_df_filtered['intersection'] = merged_df_filtered.apply(lambda x: x['pred_labels'].intersection(x['human_annotations']), axis=1)
merged_df_filtered['intersection_len'] = merged_df_filtered['intersection'].apply(len)
merged_df_filtered['correct'] = merged_df_filtered['intersection_len'] > 0

# Print the accuracy
print(merged_df_filtered['correct'].sum() / len(dropped_df_filtered))
print(f"Shape of filtered_df: {merged_df_filtered.shape}")
print("------------------")

0.8325123152709359
Shape of filtered_df: (203, 5)
------------------


### Remove sports


In [19]:
# merged_df_filtered.sample(5)

In [20]:

no_sports_csv = pd.read_csv('../notebooks/sample_no_sports.csv')
no_sports_uuids = no_sports_csv['id'].tolist()

# Filter out rows where the model prediction is 'None'
filtered_df = pred_df[pred_df['pred_labels'] != {'none'}]

# Perform the same calculation on the filtered DataFrame
dropped_df_filtered = filtered_df['pred_labels'].dropna()
merged_df_filtered = pd.merge(dropped_df_filtered, human_df, left_index=True, right_index=True)
print(merged_df_filtered.shape)

# Filter out rows where the UUID is not in no_sports_uuids
merged_df_filtered_no_sports = merged_df_filtered[merged_df_filtered.index.isin(no_sports_uuids)].copy()
print(merged_df_filtered_no_sports.shape)

merged_df_filtered_no_sports['intersection'] = merged_df_filtered_no_sports['pred_labels'].combine(
    merged_df_filtered_no_sports['human_annotations'], 
    lambda x, y: x.intersection(y)
)
merged_df_filtered_no_sports['intersection_len'] = merged_df_filtered_no_sports['intersection'].apply(len)
merged_df_filtered_no_sports['correct'] = merged_df_filtered_no_sports['intersection_len'] > 0

# Print the accuracy
print(merged_df_filtered_no_sports['correct'].sum() / len(merged_df_filtered_no_sports))
print("------------------")


(203, 2)
(188, 2)
0.824468085106383
------------------


In [21]:
from IPython.display import display, HTML
from PIL import Image
from io import BytesIO
import base64

def image_base64(im):
    if isinstance(im, str):
        im = Image.open(im)
    with BytesIO() as buffer:
        im.save(buffer, 'jpeg')
        return base64.b64encode(buffer.getvalue()).decode()

In [22]:
# merged_df = pred_df.merge(human_df, left_index=True, right_index=True, how="inner")

In [23]:
# merged_df['image_url'] = merged_df['image_url'].str.replace('data_pixtral_llava/images', '')

# img_paths = merged_df['image_url'].apply(lambda x: "../data_pixtral_llava/images/" + x.split('/')[-1]).tolist()
# # print(img_paths)
# imgStrs = [image_base64(img_path) for img_path in img_paths]

# merged_df['Image'] = [f'<img width=400 src="data:image/png;base64,{imgStr}">' for imgStr in imgStrs]
# merged_df.drop(columns=['image_url'], inplace=True)
# merged_df.to_html("../data_pixtral_llava/pixtral_anno_multi_frames_val_set.html", escape=False)

In [24]:
# sample_incorrect = merged_df[merged_df["intersection_len"] == 0].sample(4)
# plot_image(sample_incorrect['image_url'], sample_incorrect['merged_labels'], sample_incorrect['normal_multiple_frames_frame-list'])

In [25]:
# plot_image(sample_incorrect['image_url'], sample_incorrect['merged_labels'], sample_incorrect['frame-name'])

### Get gender statistics

In [26]:
anno_df.sample(2)

,mini_multiple_frames_frames-list,mini_multiple_frames_reason,prompt_entity_entity-name,prompt_entity_entity-gender,prompt_entity_sentiment,prompt_entity_sentiment-reason,image_url
uuid,,,,,,,
585010c9-18c5-4e3b-9218-f9759ef53c55,"{none, quality of life}","The image shows a recreational boat on a lake,...",Boat,None,neutral,The image shows a boat on a lake with no visib...,data_pixtral_llava/images/585010c9-18c5-4e3b-9...
5cabf216-14a1-4d38-bcf9-9e65b6d96afc,"{policy prescription and evaluation, none}",The image shows a man in a suit speaking at a ...,Kirby Smart,male,positive,The individual is standing confidently at a po...,data_pixtral_llava/images/5cabf216-14a1-4d38-b...


In [34]:
anno_df['prompt_entity_sentiment'] = anno_df['prompt_entity_sentiment'].str.lower()
anno_df['prompt_entity_sentiment'].value_counts()

neutral     185
positive     89
negative     22
none          2
Name: prompt_entity_sentiment, dtype: int64

In [29]:
anno_df['prompt_entity_entity-gender'].value_counts()

None            184
male             78
female           30
Non-binary        3
male, female      1
Male              1
Various           1
Name: prompt_entity_entity-gender, dtype: int64

In [31]:
# show all 'anno_df['prompt_entity_entity-gender']' where value is Non-binary
anno_df[anno_df['prompt_entity_entity-gender'] == 'Non-binary']

,mini_multiple_frames_frames-list,mini_multiple_frames_reason,prompt_entity_entity-name,prompt_entity_entity-gender,prompt_entity_sentiment,prompt_entity_sentiment-reason,image_url
uuid,,,,,,,
d92024bc-81b6-4b8f-a4d1-8824b9abc276,"{public opinion, quality of life}",The image shows a homeless encampment with peo...,Homeless individuals,Non-binary,Neutral,The image shows homeless individuals in an urb...,data_pixtral_llava/images/d92024bc-81b6-4b8f-a...
69249b2f-dd5e-4590-b92d-3778c18bc888,"{none, quality of life}",The image shows people resting in what appears...,Homeless individuals,Non-binary,Neutral,The image depicts homeless individuals resting...,data_pixtral_llava/images/69249b2f-dd5e-4590-b...
29a18385-47ff-4083-8f24-4c99f47ea6ab,{none},The image depicts people walking and skateboar...,Skateboarders,Non-binary,Neutral,The image shows individuals skateboarding in a...,data_pixtral_llava/images/29a18385-47ff-4083-8...


In [32]:
anno_df['image_url'] = anno_df['image_url'].str.replace('data_pixtral_llava/images', '')

img_paths = anno_df['image_url'].apply(lambda x: "../data_pixtral_llava/images/" + x.split('/')[-1]).tolist()
# print(img_paths)
imgStrs = [image_base64(img_path) for img_path in img_paths]

anno_df['Image'] = [f'<img width=400 src="data:image/png;base64,{imgStr}">' for imgStr in imgStrs]
anno_df.drop(columns=['image_url'], inplace=True)
anno_df.to_html("../data_pixtral_llava/pixtral_anno_multi_frames_val_set.html", escape=False)